In [1]:
# 1. Import and install requirements
from logger import logger
from util.database import Database
from util.scraper import Scraper
import json
import itertools
import chromedriver_autoinstaller

database = Database("sqlite:///data/pdga_data.db")
scraper = Scraper()

chromedriver_autoinstaller.install()

/Users/tobiasbeidlershenk/dev/repos/pdga-rating-bot/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[01/Nov/2024 15:01:59] INFO - Chromedriver is already installed.


'/Users/tobiasbeidlershenk/dev/repos/pdga-rating-bot/venv/lib/python3.9/site-packages/chromedriver_autoinstaller/130/chromedriver'

In [ ]:
# 2. Build list of courses and write to JSON
courses = scraper.get_courses_from_dgscene()
course_names = {
    course: scraper.get_readable_course_name(course) for course in courses
}
with open('data/course_names.json', 'w') as f:
    json.dump(course_names, f, indent=4)

In [ ]:
# 3. Build list of events and write to JSON
with open('data/course_names.json') as f:
    course_names: dict = json.load(f)

with open('data/course_events.json') as f:
    course_events: dict = json.load(f)

for i, course in enumerate(course_names):
    if course in course_events:
        logger.info(f'Skipping {course} (already scraped)...')
        continue

    logger.info(f'Fetching event {i}/{len(course_names)}')
    course_events[course] = scraper.get_all_sanctioned_events(course)
    logger.info(course_events[course])

    # periodically save ratings to file
    with open('data/course_events.json', 'w') as f:
        json.dump(course_events, f, indent=4)

logger.info("Done")

In [ ]:
# 4. Fetch ratings for each event and load into DB
try:
    logger.info('Fetching ratings...')
    
    with open('data/course_names.json') as f:
        course_names: dict = json.load(f)
    with open('data/course_events.json') as f:
        course_events: dict = json.load(f)
    
    for i, course in enumerate(course_events):
        events = course_events[course]
        rounds = []

        for j, event in enumerate(events):
            event_id = event['event_id']
            if database.event_exists(event_id):
                logger.info(f'Skipping {event_id} (already scraped)...')
                continue

            course_ratings = scraper.get_round_ratings_for_tournament(event_id)
            rounds.extend(course_ratings)
            logger.info(f'Event {j+1}/{len(events)} - Course {i+1}/{len(course_events)}')

        data = {
            'course_name': course,
            'readable_course_name': course_names[course],
            'events': course_events[course],
            'rounds': rounds
        }
        database.insert_course_data(data)
        

except BaseException as e:
    logger.info(f'Error fetching ratings: {e.with_traceback()}')
except KeyboardInterrupt as e:
    logger.info(f'Error fetching ratings: {e.with_traceback()}')

scraper.cleanup()
logger.info("Done")

In [2]:
from decimal import Decimal
import sys
from fuzzywuzzy import fuzz, process
import numpy as np
from collections import Counter
from itertools import groupby

from models.round import group_comparable_rounds, Layout, Round

def get_ratings(course_name: str, layout_name: str, score: int) -> dict:
    rounds = database.query_all_course_rounds(course_name)
    if len(rounds) == 0:
        all_course_names = [course.readable_course_name for course in database.query_all_courses()]
        scored_course_names: tuple[str, int] = process.extractBests(course_name, all_course_names, scorer=fuzz.partial_ratio, score_cutoff=0, limit=5)
        similar_course_names = [course for course, _ in scored_course_names]
        sys.exit(f"No courses found for course: '{course_name}'. \nDid you mean: {', '.join(similar_course_names)}")

    all_layout_names = set([round.layout_name for round in rounds])
    scored_layouts: tuple[str, int] = process.extractBests(layout_name, all_layout_names, scorer=fuzz.partial_token_sort_ratio, score_cutoff=0, limit=10)
    best_layout_score = scored_layouts[0][1]
    if best_layout_score < 75:
        similar_layout_names = [layout for layout, _ in scored_layouts]
        sys.exit(f"No rounds found for layout: '{layout_name}'. \nDid you mean: {', '.join(similar_layout_names)}")

    # if layout_name.lower() in [x.lower() for x in all_layout_names]:
    #     scored_layout_names = [layout_name]
    # else:
    matching_layout_names = [layout for layout, _ in process.extractBests(layout_name, all_layout_names, scorer=fuzz.partial_token_sort_ratio, score_cutoff=75, limit=100)]
    matching_rounds = [round for round in rounds if round.layout_name in matching_layout_names]
    grouped_layouts = group_comparable_rounds(matching_rounds)

    print("1st match: ")
    print(grouped_layouts[0])
    print()
    print("2nd match: ")
    print(grouped_layouts[1])
    print()
    print("3rd match: ")
    print(grouped_layouts[2])
    print()

    option_chosen = int(input("Enter the best match (1,2,3): "))
    chosen_layout = grouped_layouts[option_chosen-1]

    print(f'Rating: {chosen_layout.score_rating(score)}')

/Users/tobiasbeidlershenk/dev/repos/pdga-rating-bot/venv/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [6]:
course_name = "Foxwood"
layout_name = "Gold FPO"
score = 0

rounds = database.query_all_course_rounds(course_name)
print(set([round.layout_name for round in rounds]))

get_ratings(course_name, layout_name, score)
# print(f'Course distances: {data["chosen_layout"].layout_hole_distances} : {data["chosen_layout"].layout_total_distance}')
# print(f'Layout Par: {data["chosen_layout"].layout_par}')
# print(f'With {len(data["chosen_rounds"])} rounds')
# print(f'Using {set([x.layout_name for x in data["chosen_rounds"]])}')
# print(data['par_rating_mean'] - (score * data['stroke_value_median']))

{'FWO 2024 - White', 'FWO - 2022 - White', 'Foxwood Red 2021 - AM', 'Big White Short - Foxwood Open', 'FWO 2023 - White', 'FWO 2024 - Gold Pro Mixed Open and MA1', 'Big Red Long - Foxwood Open', 'Big Red Long - Foxwood Open Rd3', 'Foxwood White 2021', 'Big Red Short - Foxwood Open Rd3', 'Foxwood Red 2021', 'Big White Long - Foxwood Open', 'Big Red Short - Foxwood Open', 'Foxwood White 2021 - AM', 'FOW 2023 - Gold', 'FWO 2024 - Gold Pro FPO', 'FWO - 2022 - Gold'}
1st match: 
Hole distances: 1: 760, 2: 325, 3: 380, 4: 265, 5: 620, 6: 250, 7: 500, 8: 320, 9: 315, 10: 375, 11: 345, 12: 790, 13: 400, 14: 875, 15: 255, 16: 275, 17: 630, 18: 710
Total distance: 8390, Par: 62
Calculated using 15 rounds

2nd match: 
Hole distances: 1: 770, 2: 325, 3: 380, 4: 265, 5: 615, 6: 255, 7: 500, 8: 270, 9: 330, 10: 365, 11: 350, 12: 940, 13: 400, 14: 895, 15: 270, 16: 275, 17: 630, 18: 710
Total distance: 8545, Par: 63
Calculated using 15 rounds

3rd match: 
Hole distances: 1: 770, 2: 325, 3: 380, 4: 28